** 02805 Social graphs and interactions **

# Basic Stats

In [105]:
# IPython global cell magic
%reset
%matplotlib inline

# import all necessary packages
import bs4 # HTML parser
from collections import Counter, OrderedDict # counting elements and ordering keys in dictionaries
import community # python-louvain package
from __future__ import division # all numbers are float
import datetime # handle date objects
import dateparser # parse any (also foreign) date format to object: https://pypi.python.org/pypi/dateparser
import itertools # iterators for efficient looping
import json # JSON parser
import math # math operations
from matplotlib import pyplot as plt # plotting figures
import mwparserfromhell # parse MediaWiki syntax: https://github.com/earwig/mwparserfromhell
from nameparser import HumanName # parse a human name
import networkx as nx # networks creation library
import nltk # natural language processing
import numpy as np # scientific computing
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
import operator # efficient operator functions
import os # operating system operations, e.g.: with files and folders
import pandas as pd # use easy-to-use data frames for data analysis
import pickle # python data structures as files
from pprint import pprint # print data structures prettier
import re # regex
import requests # request URL content
import urllib # handle special URL chars

# make working directory
directory = os.getcwd() + '/companies'
if not os.path.exists(directory):
    os.makedirs(directory)

# files
ex1_fdat = directory + '/extraction1_data.pkl'
ex2_fdat = directory + '/extraction2_data.pkl'
ex3_tmp_fdat = directory + '/tmp_extraction3_data.pkl'
ex3_fdat = directory + '/extraction3_data.pkl'
merged = directory + '/merged_data.pkl'
extraction_csv = directory + '/company_data.csv'

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


## Load Dataset

In [126]:
# load data from all extractions
companies = dict()
if os.path.isfile(merged):
    with open(merged, 'rb') as f:
        companies = pickle.load(f)

# TODO: remove this
if os.path.isfile(ex1_fdat):
    with open(ex1_fdat, 'rb') as f:
        companies = pickle.load(f)

In [168]:
# convert from dict into dataframe
comp_df = pd.DataFrame.from_dict(companies, orient='index')
comp_df

,location_city,links,wiki_url,name,wiki_name,name_url_quoted,wiki_api_url,is_company,wiki_page_id,location_country,...,type,industry,key_people,founded,products,location,logo,defunct,parent,subsid
&pizza,NaN,NaN,https://en.wikipedia.org/w/index.php?title=%26...,NaN,&pizza,%26pizza,NaN,NaN,47858476,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
+Beryll,NaN,"{Henri Bendel, Fred Segal, Los Angeles Times}",https://en.wikipedia.org/w/index.php?title=%2B...,+Beryll,+Beryll,%2BBeryll,https://en.wikipedia.org/w/api.php?&action=par...,True,13860681,NaN,...,private,Fashion accessories,Sigmar Berg – Chief Designer,"Santa Monica, California (2006)","luxury goods, designer sunglasses",NaN,"{u'wiki_raw_code': u'Beryll logo.jpg', u'wiki_...",NaN,NaN,NaN
...instore,NaN,"{Home Bargains, Heron Foods, Tesco, Shoprite (...",https://en.wikipedia.org/w/index.php?title=......,...instore,...instore,...instore,https://en.wikipedia.org/w/api.php?&action=par...,True,9291975,NaN,...,Retail,NaN,NaN,2003,NaN,"Huddersfield, England, United Kingdom",{u'wiki_raw_code': u'[[File:Instore-logo.png]]...,2009,NaN,NaN
01 Communique,NaN,{},https://en.wikipedia.org/w/index.php?title=01+...,01 Communique,01 Communique,01+Communique,https://en.wikipedia.org/w/api.php?&action=par...,True,15244876,NaN,...,Public Listed Company (),Software,"Andrew Cheung, President",1992,"Remote Access Software, Online Help Desk Suppo...","Mississauga, Ontario, Canada",{u'wiki_raw_code': u'[[File:01 Communique Logo...,NaN,NaN,NaN
01 Distribution,Rome,{RAI},https://en.wikipedia.org/w/index.php?title=01+...,'''01 Distribution''',01 Distribution,01+Distribution,https://en.wikipedia.org/w/api.php?&action=par...,True,46724099,Italy,...,public company,"films, animation","[{u'last': u'Del Brocco', u'suffix': u'', u'ti...",21 June 2001,NaN,NaN,NaN,NaN,NaN,NaN
07th Expansion,NaN,{Alchemist (company)},https://en.wikipedia.org/w/index.php?title=07t...,07th Expansion,07th Expansion,07th+Expansion,https://en.wikipedia.org/w/api.php?&action=par...,True,5936289,NaN,...,NaN,"[Sound novels, Video game industry, Interactiv...",NaN,NaN,"[Higurashi no Naku Koro ni, Umineko no Naku Ko...",Japan,NaN,NaN,NaN,NaN
0verflow,"Kanda, Chiyoda , Tokyo",{},https://en.wikipedia.org/w/index.php?title=0ve...,0verflow,0verflow,0verflow,https://en.wikipedia.org/w/api.php?&action=par...,True,10387049,Japan,...,Private subsidiary,"[Interactive entertainment, Brand novelties]","[{u'last': u'Ōnuma', u'suffix': u'', u'title':...",1997,"[School Days, Summer Days, Cross Days]",NaN,"{u'wiki_raw_code': u'0verflowlogo.png', u'wiki...",NaN,Stack Ltd.,NaN
1-2-3 (fuel station),NaN,{Statoil Fuel & Retail},https://en.wikipedia.org/w/index.php?title=1-2...,1-2-3,1-2-3 (fuel station),1-2-3+%28fuel+station%29,https://en.wikipedia.org/w/api.php?&action=par...,True,14018485,NaN,...,Fuel station,NaN,NaN,5 December 2000,NaN,NaN,NaN,NaN,NaN,NaN
1-800 Contacts,NaN,NaN,https://en.wikipedia.org/w/index.php?title=1-8...,NaN,1-800 Contacts,1-800+Contacts,NaN,NaN,4613366,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1-800-FREE-411,NaN,"{Google, Liberty Media, Tellme Networks, March...",https://en.wikipedia.org/w/index.php?title=1-8...,Marchex,1-800-FREE-411,1-800-FREE-411,https://en.wikipedia.org/w/api.php?&action=par...,True,18480351,NaN,...,Public,Telecommunications,"Russell C. Horowitz, CEO","2005 in Burlington, MA",1-800-FREE411 directory service,"Seattle, WA, U.S.","{u'wiki_raw_code': u'800free411.gif', u'wiki_f...",NaN,Liberty Media (9%),NaN


## Basic Cleaning

### Name Conversion from List

For example Chinese companies the name is a list and also contains the Chinese name.

In [142]:
convert_names = comp_df['name'].tolist()
for idx, ele in enumerate(convert_names):
    if isinstance(ele, list):
        # some lists are empty, then replace name with wiki_name
        if not ele[0]:
            convert_names[idx] = comp_df.index[idx]
        else:
            convert_names[idx] = ele[0]
    else:
        convert_names[idx] = ele

comp_df['name'] = convert_names

### Companies with missing name

Name misses should not occur because the wikipedia link name is taken if it is missing in the company infobox.

In [143]:
c_w_name = comp_df.dropna(subset=['name'])
c_w_name

,location_city,links,wiki_url,name,wiki_name,name_url_quoted,wiki_api_url,is_company,wiki_page_id,location_country,...,type,industry,key_people,founded,products,location,logo,defunct,parent,subsid
+Beryll,NaN,"{Henri Bendel, Fred Segal, Los Angeles Times}",https://en.wikipedia.org/w/index.php?title=%2B...,+Beryll,+Beryll,%2BBeryll,https://en.wikipedia.org/w/api.php?&action=par...,True,13860681,NaN,...,private,Fashion accessories,Sigmar Berg – Chief Designer,"Santa Monica, California (2006)","luxury goods, designer sunglasses",NaN,"{u'wiki_raw_code': u'Beryll logo.jpg', u'wiki_...",NaN,NaN,NaN
...instore,NaN,"{Home Bargains, Heron Foods, Tesco, Shoprite (...",https://en.wikipedia.org/w/index.php?title=......,...instore,...instore,...instore,https://en.wikipedia.org/w/api.php?&action=par...,True,9291975,NaN,...,Retail,NaN,NaN,2003,NaN,"Huddersfield, England, United Kingdom",{u'wiki_raw_code': u'[[File:Instore-logo.png]]...,2009,NaN,NaN
01 Communique,NaN,{},https://en.wikipedia.org/w/index.php?title=01+...,01 Communique,01 Communique,01+Communique,https://en.wikipedia.org/w/api.php?&action=par...,True,15244876,NaN,...,Public Listed Company (),Software,"Andrew Cheung, President",1992,"Remote Access Software, Online Help Desk Suppo...","Mississauga, Ontario, Canada",{u'wiki_raw_code': u'[[File:01 Communique Logo...,NaN,NaN,NaN
01 Distribution,Rome,{RAI},https://en.wikipedia.org/w/index.php?title=01+...,'''01 Distribution''',01 Distribution,01+Distribution,https://en.wikipedia.org/w/api.php?&action=par...,True,46724099,Italy,...,public company,"films, animation","[{u'last': u'Del Brocco', u'suffix': u'', u'ti...",21 June 2001,NaN,NaN,NaN,NaN,NaN,NaN
07th Expansion,NaN,{Alchemist (company)},https://en.wikipedia.org/w/index.php?title=07t...,07th Expansion,07th Expansion,07th+Expansion,https://en.wikipedia.org/w/api.php?&action=par...,True,5936289,NaN,...,NaN,"[Sound novels, Video game industry, Interactiv...",NaN,NaN,"[Higurashi no Naku Koro ni, Umineko no Naku Ko...",Japan,NaN,NaN,NaN,NaN
0verflow,"Kanda, Chiyoda , Tokyo",{},https://en.wikipedia.org/w/index.php?title=0ve...,0verflow,0verflow,0verflow,https://en.wikipedia.org/w/api.php?&action=par...,True,10387049,Japan,...,Private subsidiary,"[Interactive entertainment, Brand novelties]","[{u'last': u'Ōnuma', u'suffix': u'', u'title':...",1997,"[School Days, Summer Days, Cross Days]",NaN,"{u'wiki_raw_code': u'0verflowlogo.png', u'wiki...",NaN,Stack Ltd.,NaN
1-2-3 (fuel station),NaN,{Statoil Fuel & Retail},https://en.wikipedia.org/w/index.php?title=1-2...,1-2-3,1-2-3 (fuel station),1-2-3+%28fuel+station%29,https://en.wikipedia.org/w/api.php?&action=par...,True,14018485,NaN,...,Fuel station,NaN,NaN,5 December 2000,NaN,NaN,NaN,NaN,NaN,NaN
1-800-FREE-411,NaN,"{Google, Liberty Media, Tellme Networks, March...",https://en.wikipedia.org/w/index.php?title=1-8...,Marchex,1-800-FREE-411,1-800-FREE-411,https://en.wikipedia.org/w/api.php?&action=par...,True,18480351,NaN,...,Public,Telecommunications,"Russell C. Horowitz, CEO","2005 in Burlington, MA",1-800-FREE411 directory service,"Seattle, WA, U.S.","{u'wiki_raw_code': u'800free411.gif', u'wiki_f...",NaN,Liberty Media (9%),NaN
1-800-Flowers,NaN,"{CompuServe, AOL, Comtex, Martha Stewart Livin...",https://en.wikipedia.org/w/index.php?title=1-8...,"1-800-Flowers.com, Inc.",1-800-Flowers,1-800-Flowers,https://en.wikipedia.org/w/api.php?&action=par...,True,4048045,NaN,...,Public,Retail,"[{u'last': u'McCann', u'suffix': u'', u'title'...",1976,"Flowers, Gift Baskets, Plants, Gourmet Food, C...","Carle Place, New York, United States","{u'wiki_raw_code': u'1-800-flowers com.svg', u...",NaN,NaN,NaN
1-800-GOT-JUNK?,NaN,"{HSBC Bank Canada, Teradici, The Pantry (resta...",https://en.wikipedia.org/w/index.php?title=1-8...,1-800-GOT-JUNK?,1-800-GOT-JUNK?,1-800-GOT-JUNK%3F,https://en.wikipedia.org/w/api.php?&action=par...,True,5579899,NaN,...,Private Company,Junk Removal,"Brian Scudamore, Founder and CEO",1989-11-25 00:00:00,Junk Removal Services,"Vancouver, 

### Companies with duplicate names

Can come from users that put company profiles on their pages, compare:
* https://en.wikipedia.org/wiki/Amgen
* https://en.wikipedia.org/wiki/User:Gjanko/sandbox

In [146]:
# find duplicate company names
c_dupl = pd.concat(g for _, g in comp_df.groupby('name') if len(g) > 1)
print "Found", len(c_dupl['name']), "duplicates:"
print c_dupl['name']

Abrazo Scottsdale Campus                                       Abrazo Scottsdale Campus
Abrazo Scottsdale Campus Arizona                               Abrazo Scottsdale Campus
Al Marjan Island                                                       Al Marjan Island
Al Marjan Island LLC                                                   Al Marjan Island
Alchemy (company)                                                               Alchemy
Alchemy Boulders                                                                Alchemy
Amplify (company)                                                               Amplify
Amplify (distributor)                                                           Amplify
Andersen Tax                                                           Andersen Tax LLC
Andersen Tax LLC                                                       Andersen Tax LLC
Army and Air Force Exchange Service                 Army and Air Force Exchange Service
Army and Air Force Motion Pictur

In [179]:
# add extra column adding the length of the link set
s_list = [
    'Yamaha Corporation',
    'List of Yamaha guitars',
    'Yamaha electric guitar models'
]
for s in s_list:
    print s, len(comp_df.loc[s, 'links'])

Yamaha Corporation 332
List of Yamaha guitars 1
Yamaha electric guitar models 5


In [148]:
# sort after multiple values, sorts the columns with least NaN values to top
sort_cols = ['wiki_page_id', 'num_employees']
c_dupl.sort_values(by=sort_cols, ascending=True)
c_dupl[['wiki_page_id', 'name', 'num_employees']]

TypeError: unhashable type: 'set'

In [68]:
# now drop the one's with duplcate names
c_dupl.drop_duplicates('name', inplace=True)
c_dupl

,location_city,links,wiki_url,wiki_raw,wiki_name,name_url_quoted,wiki_api_url,all_links,wiki_page_id,location_country,...,industry,key_people,founded,products,location,logo,type,defunct,parent,subsid
Greka Energy,NaN,"{Occidental Petroleum, Conoco}",https://en.wikipedia.org/w/index.php?title=Gre...,{{Infobox company \n| name = Greka \n| logo ...,Greka Energy,Greka+Energy,https://en.wikipedia.org/w/api.php?&action=par...,"[Aera Energy LLC, Asphalt, California, Canada,...",23796429,NaN,...,"[Oil and petroleum extraction and refining, Mi...","[{u'last': u'Grewal', u'suffix': u'', u'title'...","[1999, (Saba; Horizontal Ventures)]","[Oil, Petroleum, Asphalt, Emulsion]","Santa Maria, California, U.S.","{u'wiki_raw_code': u'Greka Logo Flag.jpg', u'w...",Private,NaN,NaN,"Greka Oil & Gas, Alexi Realty, Santa Maria Ref..."
Nintendo Integrated Research & Development,Kyoto,"{Nintendo Network Service Database, Spike Chun...",https://en.wikipedia.org/w/index.php?title=Nin...,{{Infobox company\n| name = Ninten...,Nintendo Integrated Research & Development,Nintendo+Integrated+Research+%26+Development,https://en.wikipedia.org/w/api.php?&action=par...,"[1-UP Studio, Affiliate (commerce), Arm Wrestl...",5444899,Japan,...,Video games,"[{u'last': u'Takeda', u'suffix': u'', u'title'...",2003-09-30 00:00:00,Various Nintendo video game consoles,NaN,{u'wiki_raw_code': u'File:Nintendo IRD logo.pn...,Division,2015-09-16 00:00:00,Nintendo,NaN
Open Systems International,NaN,{},https://en.wikipedia.org/w/index.php?title=Ope...,{{Infobox company \n| name = OSI\n| logo =...,Open Systems International,Open+Systems+International,https://en.wikipedia.org/w/api.php?&action=par...,"[Chief executive officer, Distribution managem...",19411647,NaN,...,Utility Automation,"[{u'last': u'Hoveida', u'suffix': u'', u'title...",1992,NaN,"Minneapolis, Minnesota",{u'wiki_raw_code': u'File:Open_Systems_Interna...,Private,NaN,NaN,NaN
Great Western Bank (1907–present),NaN,{National Australia Bank},https://en.wikipedia.org/w/index.php?title=Gre...,{{Infobox company |\n| name = Great Western ...,Great Western Bank (1907–present),Great+Western+Bank+%281907%E2%80%93present%29,https://en.wikipedia.org/w/api.php?&action=par...,"[Arizona, Asset, Colorado, Financial Services,...",10669154,NaN,...,Financial Services,"Ken Karels, President & CEO",1935-11-25 00:00:00,Financial Services,"Sioux Falls, South Dakota, United States",{u'wiki_raw_code': u'Great Western Bank (moder...,Public,NaN,Great Western Bancorp Inc.,NaN
Meyer Turku,Turku,"{United Shipbuilding Corporation, Meyer Werft,...",https://en.wikipedia.org/w/index.php?title=Mey...,{{Infobox company\n| name = Meyer Tu...,Meyer Turku,Meyer+Turku,https://en.wikipedia.org/w/api.php?&action=par...,"[Aker ASA, Aker Arctic, Aker Yards, Allan Staf...",48975797,Finland,...,shipbuilding,Jan Meyer,1989-11-07 00:00:00,"cruise ships, cruise ferries",Perno,"{u'wiki_raw_code': u'Meyer Turku logo.jpg', u'...",osakeyhtiö,NaN,Meyer Werft GmbH,*Piikkio Works Oy\nShipbuilding Completion Oy\...
Nepal Telecom,"Bhadrakali Plaza, Kathmandu","{Chaudhary Group, Marvell Technology Group, LG...",https://en.wikipedia.org/w/index.php?title=Nep...,{{Infobox company\n| name = {{lang...,Nepal Telecom,Nepal+Telecom,https://en.wikipedia.org/w/api.php?&action=par...,"[3G, ADSL, AKM Semiconductor, Inc., ARM Holdin...",1690862,Nepal,...,Telecommunications,Budhi Prasad Acharya,1913-11-25 00:00:00,NaN,NaN,"{u'wiki_raw_code': u'Nepal Telecom.jpg', u'wik...",Public,NaN,NaN,NaN
Yamaha Corporation,"Hamamatsu, Shizuoka","{Showa Denko, DKW, Bank of Yokohama, JVC Kenwo...",https://en.wikipedia.org/w/index.php?title=Yam...,{{Redirect|Yamaha}}\n{{Infobox company\n| name...,Yamaha Corporation,Yamaha+Corporation,https://en.wikipedia.org/w/api.php?&action=par...,"[Yamaha, Shan Xin, Japan, United Kingdom, Swed...",308428,Japan,...,Conglomerate,"[{u'last': u'Torakusu Yamaha', u'suffix': u'',...",1887-10-12 00:00:00,"Musical Instruments, Audio equipment, Electron...",NaN,{u'wiki_raw_code

## Add geolocation Data

[OpenStreetMap Nominatim](https://wiki.openstreetmap.org/wiki/Nominatim) or [Google Geocoding API (V3)](https://developers.google.com/maps/documentation/geocoding/start) is used to resolve GPS coordinates to places. Uses the Python library [geopy](https://pypi.python.org/pypi/geopy/1.11.0).

In [100]:
# show how many companies have an address
def get_location(row, geolocators, _type=None):
    
    # check from top to bottom for location
    loc_str = None
    row_to_check = ['location', 'location_city', 'location_country']
    for r in row_to_check:
        if not pd.isnull(row[r]):
            loc_str = r
    if not loc_str:
        return loc_str

    for g in geoloators:
        loc = g.geocode(loc_str, timeout=10)
        if loc and _type == 'location_gps':
            return (loc.latitude, loc.longitude)
        elif loc:
            return loc.address
    return None

In [101]:
#c_dat['location_gps'] = df.apply(lambda row: get_location(row), axis=1)
# initialize the different geolocator APIs
g_osm = geopy.geocoders.Nominatim()
g_google = geopy.geocoders.GoogleV3() # can also be with g_api_key
geolocators = [g_osm, g_google]

# applies gets location function for each row
c_dat.apply(lambda row: get_location(row, geolocators), axis=1)
# return the GPS coordinates instead
#c_dat[c_dat.apply(lambda row: get_location(row, geolocators, 'location_gps'), axis=1)]

KeyboardInterrupt: 

## Save Data

In [16]:
# specify the type for each field
# TODO: location_gps 
types = {
    # when first link is crawled
    'wiki_name': unicode,
    'wiki_url': str,
    'name_url_quoted': str,
    
    # when WIKI API is crawled
    'wiki_page_id': int,
    'wiki_api_url': str,
    'all_links': list,
    'links': set,
    'is_company': bool,
    'wiki_raw': unicode,
    
    # from extraction2
    'oc_api_url': str,
    'oc_api_network_url': str,
    
    # added only when Infobox company exists or fields from OpenCorporates
    # not all fields always exist, they are NaN in the resulting DataFrame
    'name': unicode, 
    'type': unicode, 
    'founded': datetime.datetime, 
    'defunct': datetime.datetime, 
    'location': unicode, # 
    'location_city': unicode, 
    'location_country': unicode,
    # following not in OC
    'countries': set, # added with extraction 3
    'logo': dict, 
    'key_people': list, # additonally processed with nameparser.HumanName (dict)
    'industry': list, 
    'subsid': list,
    'products': list, 
    'num_employees': int, 
    'parent': unicode, 
    'homepage': unicode
}

# save as CSV file
comp_df.to_csv(extraction_csv, encoding='utf-8', index_label='wiki_name')

In [ ]:
c_dat = pd.read_csv(extraction_csv, index_col=0, dtype=types)

## General Statistics in the Data

* Company with most links.
* Company with most employees.
* Companies by country.
* Companies by age of foundation (graph)
* Most influential people's first name (top 10)
* What is the most central company in Denmark and Europe?
* Company listed in most countries.

In [175]:
cnt = Counter()
# iterate every row that represents a company
for index, row in c_dat.iterrows():
    if isinstance(row['links'], float):
        continue
    cnt[row['name']] = len(row['links'])

print "Companies with most links:"
pprint(cnt.most_common(10))

# Italian banks have a category section: Banking in Italy (e.g.: https://en.wikipedia.org/wiki/Crediop)

Companies with most links:
[('Dexia Crediop', 2457),
 ('Banca Centropadana', 2450),
 ('Fidi Toscana', 2445),
 ('Banca Finnat', 2445),
 ('ChiantiBanca', 2445),
 ('Caript', 2441),
 ('Banca Popolare Pugliese', 2426),
 ('CR Veneto', 2425),
 ("Banca d'Alba", 2421),
 ('Banca Esperia', 2420)]


In [176]:
c_dat['num_employees'] = pd.to_numeric(c_dat['num_employees'])
c_dat.nlargest(10, 'num_employees')['num_employees']

ValueError: Unable to parse string "[c. 100 paid staff,  1,404 volunteers]" at position 6459

In [ ]:
cnt = Counter()
for index, row in c_dat.iterrows():
    if isinstance(row['num_employees'], int):
        cnt[row['name']] = int(row['num_employees'])

print "Most companies by country:"
pprint(cnt.most_common(10))